In [ ]:
# https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset?resource=download

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import emoji

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
df = pd.read_csv('../datasets/emoji_sentiment_dataset.csv')

print(df)
print(f"El dataset tiene {df.shape[0]} filas")
print(f"El dataset tiene {df.isnull().any(axis=1).sum()} filas con valores nulos")
df_clean = df.dropna()
print(f"El dataset tiene {df_clean.isnull().any(axis=1).sum()} filas con valores nulos")

In [ ]:
df_clean.groupby('category').count().plot(kind='bar')

In [ ]:

def plot_word_cloud(terms, category):
    # Definir un título basado en el valor de la categoría
    if category == 1.0:
        title = "Positivo"
    elif category == 0.0:
        title = "Neutral"
    else:
        title = "Negativo"
    
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(terms)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Nube de Palabras - Sentimiento {title}', fontsize=20)  # Añadir título aquí
    plt.axis("off")
    plt.show()

# Asegúrate de que los datos están limpios y los textos son strings
df_clean['clean_text'] = df_clean['clean_text'].astype(str)

# Generar nubes de palabras para cada categoría
for category in [-1.0, 0.0, 1.0]:
    subset = df_clean[df_clean['category'] == category]
    texts = " ".join(subset['clean_text'])
    tfidf_vector = TfidfVectorizer(stop_words='english', max_features=1000)
    X_sub = tfidf_vector.fit_transform([texts])
    max_words = {word: X_sub[0, idx] for word, idx in tfidf_vector.vocabulary_.items()}
    plot_word_cloud(max_words, category)  # Pasar también la categoría como argumento


In [ ]:
# Tokenización
tokenizer = Tokenizer(num_words=5000)  # Solo considera las 5000 palabras más frecuentes
tokenizer.fit_on_texts(df_clean['clean_text'])
sequences = tokenizer.texts_to_sequences(df_clean['clean_text'])

In [ ]:
# Padding
max_sequence_len = max([len(x) for x in sequences])
X = pad_sequences(sequences, maxlen=max_sequence_len)
y = to_categorical(np.asarray(df_clean['category'] + 1))  # Convertir -1, 0, 1 a 0, 1, 2 para categorías

In [ ]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_sequence_len),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dense(3, activation='softmax')  # 3 categorías de salida
])

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, batch_size=32, epochs=3, validation_split=0.1)

# Evaluar el modelo
print(model.evaluate(X_train, y_train))


In [ ]:
# Función para obtener el nombre Unicode de un emoji
def emoji_to_unicode_name(em):
    return emoji.demojize(em)

# Lista de emojis
new_text = '😍😍🤘🤘❤️❤️👄😎😎'

# Convertir la lista a un pandas Series
new_text_series = pd.Series(new_text)

# Aplicar la función a la Series
new_text_unicode_names = new_text_series.apply(emoji_to_unicode_name).str.replace(':',' ')


# Convertir de nuevo a lista si es necesario
new_text = new_text_unicode_names.tolist()

print(new_text)

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=max_sequence_len)

# Realizar la predicción
pred = model.predict(padded)
print(f'Predicción: {pred[0]}')  # Muestra la probabilidad de cada categoría

# Determinar la categoría más probable
predicted_category_index = np.argmax(pred[0])
categories = {-1: 'Negativo', 0: 'Neutral', 1: 'Positivo'}  # Categorías ajustadas para corresponder a tus etiquetas
predicted_category = categories[predicted_category_index - 1]  # ajustar el índice para -1, 0, 1
print(f'La categoría más probable es: {predicted_category} con una probabilidad de {pred[0][predicted_category_index]}')


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", torch_dtype=torch.bfloat16, device_map="auto")

prompt = "Generate only a list of ten relevant and popular Instagram hashtags for the following words: dog, cat, fish.  Provide only the list of hashtags, with no additional text or explanation."

# Mostrar el prompt
print("Prompt Text:")
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=1000)
output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

# Mostrar la salida de manera clara
print("Generated Text:")
print(output)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "nvidia/Llama3-ChatQA-1.5-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

messages = [
    {"role": "user", "content": "Generate a list of ten relevant and popular Instagram hashtags for the following words: football, sport and player.  Provide only the list of hashtags, with no additional text or explanation. Give only the best 10 hashtags."}
]

def get_formatted_input(messages):
    system = "System: This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context."
    instruction = "Please give a full and complete answer for the question."

    for item in messages:
        if item['role'] == "user":
            ## only apply this instruction for the first user turn
            item['content'] = instruction + " " + item['content']
            break

    conversation = '\n\n'.join(["User: " + item["content"] if item["role"] == "user" else "Assistant: " + item["content"] for item in messages]) + "\n\nAssistant:"
    formatted_input = system + "\n\n" + conversation
    
    return formatted_input

formatted_input = get_formatted_input(messages)
tokenized_prompt = tokenizer(tokenizer.bos_token + formatted_input, return_tensors="pt").to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(input_ids=tokenized_prompt.input_ids, attention_mask=tokenized_prompt.attention_mask, max_new_tokens=64, eos_token_id=terminators)

response = outputs[0][tokenized_prompt.input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


c:\Users\hugoc\anaconda3\envs\thefreeai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hugoc\anaconda3\envs\thefreeai\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.34s/it]
c:\Users\hugoc\anaconda3\envs\thefreeai\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a 

 #football #soccer #footballplayer #footballgame #footballmatch #footballfans #footballseason #footballtraining #footballteam #footballstar
